<a href="https://colab.research.google.com/github/soumya-mishra/TransferLearning/blob/main/Transfer_Learning_Notebook_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the dataset from the link provided using !wget command

In [ ]:
!wget -q https://datasetsgun.s3.amazonaws.com/upgrad/flowers-20210317T060850Z-001.zip

Unzip the downloaded file using !unzip command

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip -q flowers-20210317T060850Z-001.zip

Create a new directory 'data' to save your Train & Test data

In [ ]:
import os
os.mkdir('data1')

In [ ]:
import os
import shutil
import numpy as np
import glob   
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Input, Add,Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import  plot_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline


In [ ]:
# Where all dataset is there
data_dir = 'flowers/'

# Training data dir
training_dir = 'data1/Train'

# Test data dir
testing_dir = 'data1/Test'

# Ratio of training and testing data
train_test_ratio = 0.8 


def split_dataset_into_test_and_train_sets(all_data_dir = data_dir, training_data_dir = training_dir, testing_data_dir=testing_dir, train_test_ratio = 0.8):
    # Recreate testing and training directories
    
    if not os.path.exists(training_data_dir):
            os.mkdir(training_data_dir)

    if not os.path.exists(testing_data_dir):
            os.mkdir(testing_data_dir)               
    
    num_training_files = 0
    num_testing_files = 0


    for subdir, dirs, files in os.walk(all_data_dir):
        
        category_name = os.path.basename(subdir)
        
        # print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
          continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name
        
        # creating subdir for each sub category
        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)   

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)
            
        file_list = glob.glob(os.path.join(subdir,'*.jpg'))

        #print(os.path.join(all_data_dir, subdir))
        print(str(category_name) + ' has ' + str(len(files)) + ' images') 
        random_set = np.random.permutation((file_list))
        # copy percentage of data from each category to train and test directory
        train_list = random_set[:round(len(random_set)*(train_test_ratio))] 
        test_list = random_set[-round(len(random_set)*(1-train_test_ratio)):]

  

        for lists in train_list : 
            shutil.copy(lists, training_data_dir + '/' + category_name + '/' )
            num_training_files += 1
  
        for lists in test_list : 
            shutil.copy(lists, testing_data_dir + '/' + category_name + '/' )
            num_testing_files += 1
  

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [ ]:
split_dataset_into_test_and_train_sets()

dandelion has 1055 images
rose has 784 images
flowers has 0 images
dandelion has 1055 images
rose has 784 images
sunflower has 734 images
daisy has 769 images
tulip has 984 images
sunflower has 734 images
daisy has 778 images
tulip has 984 images
Processed 6923 training files.
Processed 1732 testing files.


In [ ]:
# Number of classes in dataset
num_classes = 6

def get_model():
    # Get base model 
    # Here we are using ResNet50 as base model | Actual softmax layer we do not have to include.
    base_model = ResNet50(weights='imagenet', include_top=False)
    
    # As we are using ResNet model only for feature extraction and not adjusting the weights
    # we freeze the layers in base model
    for layer in base_model.layers:
        layer.trainable = False
        
    # Get base model output 
    base_model_ouput = base_model.output
    
    # Adding our own layer 
    x = GlobalAveragePooling2D()(base_model_ouput)
    # Adding fully connected layer
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax', name='fcnew')(x)
    
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [ ]:
# Get the model
model = get_model()
# Compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# Summary of model
model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
_____________________________

In [ ]:
# Defining the imagedatagenerator for train and test image for pre-processing
# We don't give horizonal_flip or other preprocessing for validation data generator

image_size = 224
batch_size = 64

train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

Found 4158 images belonging to 6 classes.
Found 1559 images belonging to 6 classes.


In [ ]:
# Training the fully conncected layer for initial epochs
epochs = 5

# Training the model

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs)

Epoch 1/5
64/64 [==============================] - 99s 994ms/step - loss: 1.0664 - accuracy: 0.6187 - val_loss: 0.4447 - val_accuracy: 0.8431
Epoch 2/5
64/64 [==============================] - 63s 977ms/step - loss: 0.4381 - accuracy: 0.8505 - val_loss: 0.3472 - val_accuracy: 0.8750
Epoch 3/5
64/64 [==============================] - 64s 992ms/step - loss: 0.3529 - accuracy: 0.8845 - val_loss: 0.3033 - val_accuracy: 0.8913
Epoch 4/5
64/64 [==============================] - 63s 984ms/step - loss: 0.3135 - accuracy: 0.8992 - val_loss: 0.2787 - val_accuracy: 0.9056
Epoch 5/5
64/64 [==============================] - 63s 977ms/step - loss: 0.2671 - accuracy: 0.9078 - val_loss: 0.2638 - val_accuracy: 0.9023


In [ ]:
# More fine tuning the model
# Training the model after 140 layers
# Generally ResNet is good at extracting lower level features so we are not fine tuning initial layers
epochs = 10

split_at = 140
for layer in model.layers[:split_at]: 
  layer.trainable = False
for layer in model.layers[split_at:]: 
  layer.trainable = True
    
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate, if we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
64/64 [==============================] - 69s 1s/step - loss: 0.3943 - accuracy: 0.8795 - val_loss: 0.2181 - val_accuracy: 0.9258
Epoch 2/10
64/64 [==============================] - 65s 1s/step - loss: 0.2266 - accuracy: 0.9308 - val_loss: 0.1689 - val_accuracy: 0.9434
Epoch 3/10
64/64 [==============================] - 65s 1s/step - loss: 0.1877 - accuracy: 0.9396 - val_loss: 0.1314 - val_accuracy: 0.9564
Epoch 4/10
64/64 [==============================] - 65s 1s/step - loss: 0.1446 - accuracy: 0.9543 - val_loss: 0.1040 - val_accuracy: 0.9701
Epoch 5/10
64/64 [==============================] - 64s 1s/step - loss: 0.1172 - accuracy: 0.9658 - val_loss: 0.0862 - val_accuracy: 0.9772
Epoch 6/10
64/64 [==============================] - 65s 1s/step - loss: 0.0909 - accuracy: 0.9768 - val_loss: 0.0738 - val_accuracy: 0.9805
Epoch 7/10
64/64 [==============================] - 65s 1s/step - loss: 0.0825 - accuracy: 0.9750 - val_loss: 0.0626 - val_accuracy: 0.9831
Epoch 8/10
64/64 [==

In [ ]:
print('Training complete')

Training complete
